In [1]:
import numpy as np
import cvxpy as cvx
from copy import copy
from itertools import product
from pympc.geometry.polytope import Polytope
import matplotlib.pyplot as plt

In [2]:
def maximum_volume_box_inscribed_polytope(A, b):
    A_pos = np.maximum(np.zeros(A.shape), A)
    m, d = A.shape
    x = cvx.Variable(d, 1)
    y = cvx.Variable(d, 1)
    y_sum = sum([cvx.log(y[i,0]) for i in range(d)])
    objective = cvx.Maximize(y_sum)
    constraints = [A*x + A_pos*y <= b]
    prob = cvx.Problem(objective, constraints)
    result = prob.solve(solver=cvx.CVXOPT)
    x = np.array(x.value)
    y = np.array(y.value)
    print 'check:', np.max(A.dot(x)-b), np.max(A.dot(x+y)-b), 
    return x, x+y

In [3]:
def box_in_projection(A, b, residual_dimensions):
    dropped_dimensions = [i for i in range(A.shape[1]) if i not in residual_dimensions]
    A_D = A[:,residual_dimensions]
    A_N = A[:,dropped_dimensions]
    A_D_pos = np.maximum(np.zeros(A_D.shape), A_D)
    x = cvx.Variable(A_D.shape[1], 1)
    y = cvx.Variable(A_D.shape[1], 1)
    u = cvx.Variable(A_N.shape[1], 1)
    y_sum = sum([cvx.log(y[i,0]) for i in range(A_D.shape[1])])
    objective = cvx.Maximize(y_sum)
    constraints = []
    for i in range(A_D.shape[0]):
        constraints.append(A_D[i,:]*x + A_D_pos[i,:]*y + A_N[i,:]*u <= b[i,:])
    prob = cvx.Problem(objective, constraints)
    result = prob.solve(solver=cvx.CVXOPT)
    x = np.array(x.value)
    y = np.array(y.value)
    return x, x+y

In [4]:
n_var = 20
n_cons = 100
residual_dimensions = [0,1]
A = np.random.randn(n_cons, n_var)
b = np.random.rand(n_cons, 1)
poly = Polytope(A, b).assemble()

lb, ub = maximum_volume_box_inscribed_polytope(A, b)
box = Polytope.from_bounds(lb, ub)
box.assemble()
poly.plot(facecolor='b')
box.plot(facecolor='r')
plt.show()

In [5]:
lb, ub = box_in_projection(A, b, residual_dimensions)
box = Polytope.from_bounds(lb, ub)
box.assemble()
poly.plot(facecolor='b')
box.plot(facecolor='r')
plt.show()

AttributeError: 'NoneType' object has no attribute 'strip'

In [ ]:
poly.plot(facecolor='b', dim_proj=residual_dimensions)
plt.show()

In [ ]:
cvx.installed_solvers()